In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# MMM(최고 월평균) 계산 함수 (예시: 2015~2023년 데이터 중 월별 평균의 최대값)
def calculate_mmm(group):
    group['Month'] = group['Date'].dt.month
    mmm = group.groupby('Month')['SST (째C)'].mean().max()
    group['MMM'] = mmm
    return group

# HotSpot 계산 및 DHW 계산 함수
def calculate_dhw(group, window=84):
    sst = group['SST (째C)'].values
    mmm = group['MMM'].values
    # HotSpot: SST - MMM
    hotspot = sst - mmm
    # HotSpot이 1°C 이상인 값만 누적
    hotspot = np.where(hotspot >= 1, hotspot, 0)
    # 12주(84일) 이동합
    dhw = []
    for i in range(len(hotspot)):
        start = max(0, i-window+1)
        # 7로 나눠서 °C-weeks로 변환
        dhw.append(hotspot[start:i+1].sum() / 7)
    group['DHW'] = dhw
    return group

# 파일 불러오기
file_path = '/content/drive/MyDrive/realistic_ocean_climate_dataset.csv'

try:
    df = pd.read_csv(file_path, encoding='cp949', parse_dates=['Date'])
except FileNotFoundError:
    df = None



Mounted at /content/drive


In [ ]:
if df is not None:
    df = df.sort_values(['Location', 'Date'])
    # MMM 계산
    df = df.groupby('Location', group_keys=False).apply(calculate_mmm)
    # DHW 계산
    df = df.groupby('Location', group_keys=False).apply(calculate_dhw, window=84)
    print(df[['Date', 'Location', 'SST (째C)', 'MMM', 'DHW']].to_string(index=False))
else:
    print('File not found. Please upload the dataset file.')


      Date           Location  SST (째C)       MMM      DHW
2015-01-14      Caribbean Sea     28.86 29.890000 0.000000
2015-03-01      Caribbean Sea     30.09 29.890000 0.000000
2015-03-14      Caribbean Sea     28.04 29.890000 0.000000
2015-04-16      Caribbean Sea     30.72 29.890000 0.000000
2015-04-22      Caribbean Sea     28.99 29.890000 0.000000
2015-04-29      Caribbean Sea     27.45 29.890000 0.000000
2015-08-26      Caribbean Sea     26.89 29.890000 0.000000
2015-09-01      Caribbean Sea     28.39 29.890000 0.000000
2016-05-22      Caribbean Sea     31.31 29.890000 0.202857
2016-06-04      Caribbean Sea     30.53 29.890000 0.202857
2016-06-17      Caribbean Sea     28.29 29.890000 0.202857
2016-09-04      Caribbean Sea     29.64 29.890000 0.202857
2016-09-11      Caribbean Sea     26.23 29.890000 0.202857
2016-09-24      Caribbean Sea     26.09 29.890000 0.202857
2016-12-18      Caribbean Sea     28.20 29.890000 0.202857
2017-03-20      Caribbean Sea     29.59 29.890000 0.2028

<ipython-input-8-a170ea777461>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Location', group_keys=False).apply(calculate_mmm)
<ipython-input-8-a170ea777461>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Location', group_keys=False).apply(calculate_dhw, window=84)


In [ ]:
# 저장 경로 설정
output_path = '/content/drive/MyDrive/Data Science 2025/DHW_NOAA.csv'

# CSV로 저장
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 성공적으로 저장되었습니다: {output_path}")


CSV 파일이 성공적으로 저장되었습니다: /content/drive/MyDrive/Data Science 2025/DHW_NOAA.csv
